In [ ]:
def run_remote(ssh, cmd, show=True):
    """원격 명령 실행 + 실시간 출력"""
    stdin, stdout, stderr = ssh.exec_command(cmd)
    for line in iter(stdout.readline, ""):
        if show:
            print(line, end="")
            sys.stdout.flush()
    err = stderr.read().decode()
    if err and show:
        print("[ERROR]", err)
    return err

def install_lamp_ftp():
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, password=pw, port=22)

    print("Ubuntu LAMP + FTP 서버 설치를 시작합니다...")

    #  시스템 패키지 업데이트
    run_remote(ssh, "apt update -y")
    run_remote(ssh, "apt upgrade -y")

    #  Apache 설치
    run_remote(ssh, "apt install -y apache2")
    run_remote(ssh, "systemctl enable apache2")
    run_remote(ssh, "systemctl start apache2")
    print("Apache 설치 및 자동 시작 완료!")

    #  사용자 웹 디렉토리 생성
    WEB_DIR="/var/www/html"                                    # 경로 변경
    run_remote(ssh, f"mkdir -p {WEB_DIR}")
    run_remote(ssh, f"chown -R www-data:www-data {WEB_DIR}")
    run_remote(ssh, f"chmod 755 {WEB_DIR}")
    print("웹 디렉토리 생성 및 권한 설정 완료!")

    #  VirtualHost 설정
    VHOST_CONF="/etc/apache2/sites-available/young.conf"       #  파일명 변경
    vhost_content = f"""
<VirtualHost *:80>
    ServerName www.young.com
    DocumentRoot {WEB_DIR}
    <Directory {WEB_DIR}>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>
"""
    # SFTP로 파일 작성
    sftp = ssh.open_sftp()
    with sftp.file(VHOST_CONF, 'w') as f:
        f.write(vhost_content)
    sftp.close()
    run_remote(ssh, f"a2ensite young.conf")                  #  파일명 변경
    run_remote(ssh, "systemctl reload apache2")
    print("VirtualHost 설정 완료!")

    #  index.html 생성
    run_remote(ssh, f'echo "qqq" > {WEB_DIR}/index.html')
    print("index.html 생성 완료!")

    #  vsftpd 설치
    run_remote(ssh, "apt install -y vsftpd")
    run_remote(ssh, "systemctl enable vsftpd")
    run_remote(ssh, "systemctl start vsftpd")
    print("vsftpd 설치 및 자동 시작 완료!")

    print(" 모든 설정 완료! Ubuntu LAMP + FTP 서버 준비 완료!")

    ssh.close()